In [171]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

In [217]:
gas = ct.Solution("test.cti")
gas.Y = np.array([0.5, 0, 0.5, 0, 0])
gas.TP = 1200, 101325
reactor = ct.IdealGasReactor(gas)
sim = ct.ReactorNet([reactor])
sim.advance(5.93e-6)
gas()


  CH4_MP1:

       temperature         3126.91  K
          pressure          264029  Pa
           density        0.217034  kg/m^3
  mean mol. weight          21.371  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      5.7833e+05        1.236e+07     J
   internal energy     -6.3821e+05       -1.364e+07     J
           entropy           15506        3.314e+05     J/K
    Gibbs function     -4.7909e+07       -1.024e+09     J
 heat capacity c_p          4011.5        8.573e+04     J/K
 heat capacity c_v          3622.5        7.742e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2    2.31807e-05      3.47084e-05         -40.1641
               H2O       0.333912          0.28148         -39.0537
               CH4       0.499109          0.37467         -35.9531
               CO2      

/home/darve/adncat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  """Entry point for launching an IPython kernel.


In [228]:
gas = ct.Solution("test.cti")
gas.Y = np.array([0.5, 0, 0.5, 0, 0])
gas.TP = 1200, 101325
reactor = ct.IdealGasReactor(gas)
sim = ct.ReactorNet([reactor])
n_t = 10000
Pt = np.zeros(n_t)
Tt = np.zeros(n_t)
Yt = np.zeros((5,n_t))
for i in range(n_t):
    sim.advance((i+1)*1e-9)
    Pt[i] = gas.P
    Tt[i] = gas.T
    Yt[:,i] = gas.Y
gas()

np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/pressure.npy', Pt)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature.npy', Tt)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/mass_fraction.npy', Yt)

In [244]:
NASA_coeffs = []
for i in range(gas.n_species):
    NASA_coeffs.append(gas.species(i).thermo.coeffs)
NASA_coeffs = np.array(NASA_coeffs)
reactants = []
orders = [] 
Af = np.zeros(gas.n_reactions)
beta = np.zeros(gas.n_reactions)
E = np.zeros(gas.n_reactions)
for i in range(gas.n_species):
    reactants.append(gas.species(i).name)
orders = np.zeros((gas.n_species, gas.n_reactions))
for i in range(gas.n_reactions):   
    Af[i] = gas.reaction(i).rate.pre_exponential_factor
    beta[i] = gas.reaction(i).rate.temperature_exponent
    E[i] = gas.reaction(i).rate.activation_energy
    for j in range(gas.n_species):
        if (gas.reactant_stoich_coeffs()[j,i] != 0):
            orders[j,i] = gas.reaction(i).orders[gas.species(j).name]

In [245]:
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/NASA_coeffs.npy', NASA_coeffs)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/molecular_weights.npy',gas.molecular_weights)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/reactants_stoich_coeffs.npy',gas.reactant_stoich_coeffs())
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/product_stoich_coeffs.npy',gas.product_stoich_coeffs())
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/reaction_orders.npy',orders)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy', Af)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy', beta)
np.save('/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy', E)